**MACHINE LEARNING PROJECT**

In [1]:
import numpy as np
import pandas as pd

In [ ]:
np.random.seed(30)
samples = 3000

**ALTITUDE** 

Here the +5000 value is used to handle the zero mean position of the exponential graph nature, it would be around 0.2 or close to zero which isn't any realistic altitude for any aviation model. So, the +5000 would ensures that we avoid clustering about the zero.

In [ ]:
altitude = np.round(np.random.exponential(10000, samples) + 5000, 2)
altitude = np.clip(altitude, 0, 50000) 

print("The List of altitude generates:\n", altitude)
print(altitude.shape)

**AIRSPEED**

In [ ]:
airspeed = np.where(
  np.random.rand(samples) > 0.5,
  np.round(np.random.normal(400, 100, samples), 2), # Dog Fight Mode 
  np.round(np.random.normal(700, 150, samples), 2) # Escape Mode      
   )

airspeed = np.clip(airspeed, 100, 1200)

**EJECTION TYPE and POSTURE OF THE PILOT**

Zero-Zero Ejection type typically means the pilot can eject safely event at 0.Knots speed and 0.ft altitude, which are equiped in modern day aircrafts.

A Conventional Ejection type means, the aircraft has to be at a minumum altitude and a minimum speed to eject safely, which was in the case of old aircraft.

In [ ]:
ejection_type = np.random.choice(["Zero-Zero", "Conventional"], samples, p = [0.7, 0.3])

**PILOT POSTURE**

It is very important for the pilot to be at the seat at a proper position so that he ejects safely taking the whole multiple of G-force in the spine. Even Slight amount of slouch could increase the risk of spine injury.

So, we take optimal Position to be 80% and Slouched position to be at 20%.

In [ ]:
pilot_posture = np.random.choice(["Optimal", "Slouched"], samples, p = [0.8, 0.2] )

**ENVIRONMENTAL CONDITIONS** 

Another major concern is Environmental Conditions:
1) Time of Day(Day/Night)
2) Weather(Clear/Rain/Strom)
3) Terrain(Land/Water/Mountain)


In [ ]:
time_of_Day = np.random.choice(["Day", "Night"], samples, p = [0.6, 0.4])

weather = np.random.choice(["Clear", "Rain", "Storm"], samples, p = [0.7, 0.2, 0.1])

terrain = np.random.choice(["Land", "Water", "Mountain"], samples, p = [0.6, 0.3, 0.1])

**TECHNICAL FACTORS**

An Important aspect for estimating this success, includes Technical fcators like:

1) System Malfunction
2) Aicraft Altitude
3) G-Force

**G-Force:** In the context of aerodynamics and aviation, g-force (or gravitational force) measures acceleration, specifically the force felt as weight during flight maneuvers, and is expressed as a multiple of Earth's gravity (1g); where g is 9.8 m/s^2


In [ ]:
system_malfunction = np.random.choice([0, 1], samples, p = [0.9, 0.1])

aircraft_altitude = np.random.choice(["Stable", "Diving"])

g_force = np.round(np.random.weibull(1.5, size=samples) * 10 + 5, 2)
g_force = np.clip(g_force, 5, 20)

**SUCCESSFUL EJECTION PROBABILITY**

In [ ]:
success_prob = (
  1.5 * (ejection_type == "Zero-Zero") +
  1.0 * (pilot_posture == "Optimal") +
  0.8 * (aircraft_altitude == "Stable") +
  1.2 * (system_malfunction == 0) -
  0.005 * altitude / 1000 - # Scaling of the altitude is done to take in kiloft which would allow the calculation to be meaningfull
  0.002 * airspeed -
  0.02 * g_force +
  np.random.normal(0, 0.05, samples)
)

ejection_success = (success_prob > 0.5).astype(int)

**INJURY LEVEL**

In [ ]:
injury_level = np.where(
  (ejection_success == 0) & (g_force > 15), "Fatal",
  np.where((g_force > 9) & (g_force < 15), "severe", 
  np.where(g_force < 9, "Minor", "None")
  ) 
)

In [ ]:
ejection_success_df = pd.DataFrame({
  "Altitude_ft": altitude,
  "Airspeed_knots": airspeed,
  "Ejection_Type": ejection_type,
  "Pilot_Posture": pilot_posture,
  "Time_of_Day": time_of_Day,
  "Weather": weather, 
  "Terrain": terrain, 
  "System_Malfunction": system_malfunction,
  "Aircraft_Altitude": aircraft_altitude,
  "G_Force": g_force,
  "Ejection_Success": ejection_success,
  "Injury_Level": injury_level     
})

ejection_success_df

In [ ]:
ejection_success_df.Ejection_Success.value_counts()

In [ ]:
ejection_success_df.to_csv("Fighter_Pilot_Ejection_Success.csv", index=False)